<a href="https://colab.research.google.com/github/mcadete/LLM_4_Biz_16/blob/main/Tech16_HW3_MarioC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# !pip uninstall llama-index llama-hub -y

Found existing installation: llama-index 0.12.19
Uninstalling llama-index-0.12.19:
  Successfully uninstalled llama-index-0.12.19
Found existing installation: llama-hub 0.0.79.post1
Uninstalling llama-hub-0.0.79.post1:
  Successfully uninstalled llama-hub-0.0.79.post1


In [12]:
!pip install llama-index llama_hub openai llama-index-vector-stores-chroma beautifulsoup4 requests markdownify llama-index-readers-github

In [7]:
from openai import OpenAI
from google.colab import userdata
import os

open_ai_key = userdata.get('open_ai_key')
client = OpenAI(api_key=open_ai_key)
os.environ["OPENAI_API_KEY"] = open_ai_key

In [14]:
import requests
from bs4 import BeautifulSoup
from markdownify import markdownify as md
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.readers.github import GithubRepositoryReader, GithubClient
import chromadb

# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = chroma_client.get_or_create_collection("k8s_docs")

# Function to scrape Kubernetes documentation
def scrape_kubernetes_docs(base_url):
    """
    Scrapes textual content from the official Kubernetes documentation webpage.
    """
    response = requests.get(base_url)
    if response.status_code != 200:
        return ""

    from bs4 import BeautifulSoup
    text_content = BeautifulSoup(response.text, "html.parser").get_text(separator=" ", strip=True)
    return text_content

# Scrape Kubernetes Documentation
print("Scraping Kubernetes documentation...")
kubernetes_docs_data = scrape_kubernetes_docs("https://kubernetes.io/docs/")

# Save Kubernetes docs to a text file
os.makedirs("data", exist_ok=True)
with open("data/kubernetes_docs.txt", "w", encoding="utf-8") as f:
    f.write(kubernetes_docs_data)

# Load Kubernetes Docs into LlamaIndex
reader = SimpleDirectoryReader(input_dir="data")
documents = reader.load_data()

# Scrape GitHub Repository using LlamaIndex's GitHub Reader
print("Scraping GitHub repository...")
github_reader = GithubRepositoryReader(
    owner="kelseyhightower",
    repo="kubernetes-the-hard-way",
    filter_directories=(".github", "docs"),  # Include only relevant directories
    filter_extensions=(".md", ".yml", ".sh", ".tf", ".yaml"),  # Include relevant file types
    verbose=True,
    concurrent_requests=10
)

# Load GitHub repository files
github_documents = github_reader.load_data()
documents.extend(github_documents)

# Create vector store index with ChromaDB
service_context = ServiceContext.from_defaults(embed_model=OpenAIEmbedding())
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create an index with all Kubernetes documents
index = VectorStoreIndex.from_documents(documents, service_context=service_context, storage_context=storage_context)


Scraping Kubernetes documentation...
Scraping GitHub repository...


TypeError: GithubRepositoryReader.__init__() got an unexpected keyword argument 'filter_extensions'

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("What command do you use to list all the running pods on a cluster?")
print(response)

In [ ]:
from re import VERBOSE
chat_engine = index.as_chat_engine(chat_mode="openai", verbose=True)
response = chat_engine.chat("how do you initialize a new cluster?")
print(response)

In [ ]:
response = chat_engine.chat("how do you find all the running workloads on a cluster?")
print(response)

In [3]:
# import requests
# from bs4 import BeautifulSoup
# from markdownify import markdownify as md
# from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
# from llama_index.embeddings.openai import OpenAIEmbedding
# from llama_index.core import StorageContext
# from llama_index.vector_stores.chroma import ChromaVectorStore
# import chromadb


# # Initialize ChromaDB
# chroma_client = chromadb.PersistentClient(path="./chroma_db")
# chroma_collection = chroma_client.get_or_create_collection("k8s_docs")

# # Function to scrape GitHub markdown content
# def scrape_github_markdown(repo_url):
#     raw_url = repo_url.replace("github.com", "raw.githubusercontent.com").replace("/tree/main", "")
#     response = requests.get(raw_url)
#     if response.status_code == 200:
#         return response.text
#     return ""

# # Function to scrape Kubernetes docs
# def scrape_kubernetes_docs(base_url):
#     response = requests.get(base_url)
#     if response.status_code != 200:
#         return ""

#     soup = BeautifulSoup(response.text, "html.parser")
#     text_content = soup.get_text(separator=" ", strip=True)
#     return text_content

# # Scrape and store data
# print("Scraping Kubernetes documentation...")
# github_data = scrape_github_markdown("https://github.com/kelseyhightower/kubernetes-the-hard-way/blob/main/README.md")
# kubernetes_docs_data = scrape_kubernetes_docs("https://kubernetes.io/docs/")

# # Save scraped content to files
# os.makedirs("data", exist_ok=True)
# with open("data/kubernetes_the_hard_way.md", "w", encoding="utf-8") as f:
#     f.write(md(github_data))

# with open("data/kubernetes_docs.txt", "w", encoding="utf-8") as f:
#     f.write(kubernetes_docs_data)

# # Load documents into LlamaIndex
# reader = SimpleDirectoryReader(input_dir="data")
# documents = reader.load_data()

# # Create vector store index with ChromaDB
# # chroma_collection = chroma_client.delete_collection("kubernetes")
# # chroma_collection = chroma_client.create_collection("kubernetes")

# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

